## 準備工作

In [1]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [35]:
#title
asins = open('../US_20171130/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
len(ASINs)

6880

In [36]:
#synopses
review = open('../US_20171130/pre/review.csv').read().split('\n')
# review[0]
Reviews=[]
for rev in review:
    a = rev.split('|')
   
    Reviews += a 
Reviews[0]
Reviews = Reviews[:-1]
len(Reviews)

6880

In [37]:
#genres
categorys = open('../US_20171130/pre/categories.csv').read().split('\n')

Category=[]
for cat in categorys:
    a = cat.split(',')
    Category += a
len(Category)

6880

In [39]:
#ranks
ranks = open('../US_20171130/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split('\t')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

6880

In [7]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [8]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [9]:
# 这里我定义了一个分词器（tokenizer）和词干分析器（stemmer），它们会输出给定文本词干化后的词集合
 
def tokenize_and_stem(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
 
def tokenize_only(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [10]:
# 非常不 pythonic，一点也不！
# 扩充列表后变成了非常庞大的二维（flat）词汇表
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews:
    allwords_stemmed = tokenize_and_stem(i) #对每个评价进行分词和词干化
    totalvocab_stemmed.extend(allwords_stemmed) # 扩充“totalvocab_stemmed”列表
 
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [11]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 139724 items in vocab_frame


In [12]:
print (vocab_frame.head())

     words
not    not
upto  upto
the    the
mark  mark
poor  poor


## Tf-idf and document similarity

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# 定义向量化参数
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.05, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))
 
%time tfidf_matrix = tfidf_vectorizer.fit_transform(Reviews) # 向量化剧情简介文本
 
print(tfidf_matrix.shape)


CPU times: user 5.96 s, sys: 46.7 ms, total: 6 s
Wall time: 6.02 s
(9620, 13)


In [70]:
print(tfidf_matrix)

  (0, 10)	0.647531612505
  (0, 3)	0.762038588791
  (1, 10)	0.490970545833
  (1, 9)	0.871176172266
  (2, 9)	0.735149382426
  (2, 2)	0.677905144927
  (3, 12)	0.81570858682
  (3, 11)	0.578463051013
  (4, 2)	0.327243200899
  (4, 12)	0.772531426092
  (4, 0)	0.544157222837
  (5, 5)	1.0
  (6, 10)	0.443634478072
  (6, 9)	0.787183243012
  (6, 12)	0.428405172456
  (7, 8)	1.0
  (8, 5)	1.0
  (9, 10)	0.388234347548
  (9, 3)	0.456888202232
  (9, 9)	0.344440736527
  (9, 2)	0.317620000773
  (9, 12)	0.374906845243
  (9, 4)	0.52957552206
  (10, 3)	0.698154864782
  (10, 9)	0.526327829567
  :	:
  (9595, 12)	0.729779609459
  (9595, 5)	0.460714989683
  (9596, 9)	0.543671699459
  (9596, 11)	0.839297970453
  (9597, 10)	0.719346009349
  (9597, 12)	0.694651940783
  (9598, 9)	1.0
  (9599, 10)	0.53224822126
  (9599, 2)	0.435439784026
  (9599, 4)	0.72601930102
  (9600, 2)	1.0
  (9601, 10)	0.914098140306
  (9601, 9)	0.405493020766
  (9603, 2)	1.0
  (9604, 2)	0.557953588702
  (9604, 7)	0.829872154524
  (9607, 9)	0.5

In [71]:
terms = tfidf_vectorizer.get_feature_names()

In [72]:
print(terms)

["'s", 'buy', 'good', 'kid', 'like', 'love', "n't", 'nice', 'price', 'product', 'qualiti', 'use', 'veri']


In [73]:
len(terms)

13

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [75]:
print(dist)

[[ -2.22044605e-16   6.82081051e-01   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  6.82081051e-01  -2.22044605e-16   3.59555375e-01 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   3.59555375e-01   0.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 ..., 
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]]


# K-means clustering

In [87]:
from sklearn.cluster import KMeans
 
num_clusters = 10
 
km = KMeans(n_clusters=num_clusters)
 
%time km.fit(tfidf_matrix)
 
clusters = km.labels_.tolist()

CPU times: user 1.45 s, sys: 9.74 ms, total: 1.46 s
Wall time: 1.46 s


In [88]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [89]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters, 'category': Category }
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster', 'category'])


In [90]:
frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

2    3641
8     910
1     838
5     757
0     726
3     690
4     622
6     493
7     480
9     463
Name: cluster, dtype: int64

In [91]:
grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
grouped.mean() # 每个聚类的平均排名（0 到 5）

cluster
0    4.187879
1    4.214021
2    3.955108
3    4.362681
4    4.326367
5    4.293395
6    4.401623
7    4.263750
8    4.169945
9    4.270194
Name: rank, dtype: float64

In [92]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
 
    for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() # 空行
    print() # 空行

Top terms per cluster:

Cluster 0: b'quality', b'good', b'product', b'very', b'useful', b"n't", b'buy', b"'s", b'price', b'nice', b'love', b'kid', b'likes',

Cluster 1: b'product', b'good', b'useful', b'buy', b"'s", b"n't", b'quality', b'very', b'likes', b'nice', b'price', b'kid', b'love',

Cluster 2: b"n't", b"'s", b'buy', b'useful', b'good', b'product', b'quality', b'very', b'nice', b'love', b'kid', b'likes', b'price',

Cluster 3: b'kid', b'love', b'good', b'product', b'very', b'useful', b'quality', b"'s", b'buy', b"n't", b'nice', b'likes', b'price',

Cluster 4: b'nice', b'product', b'very', b'good', b'quality', b'kid', b"'s", b'useful', b'love', b'buy', b"n't", b'price', b'likes',

Cluster 5: b'good', b"'s", b'useful', b'product', b'quality', b'very', b'kid', b'love', b'nice', b'buy', b"n't", b'price', b'likes',

Cluster 6: b'love', b'good', b'quality', b'product', b'very', b"'s", b'useful', b'buy', b'nice', b"n't", b'kid', b'price', b'likes',

Cluster 7: b'likes', b'good', b'very',

In [93]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
for i in range(num_clusters):
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() # 空行
    print() # 空行


Top terms per cluster:

Cluster 0 titles: B0746KMZ4W_4, B01MFFRJAX_5, B00SWURF62_4, B00SWURF62_7, B01HV9XPN6_1, B00RJE7MWE_4, B00RJE7MWE_9, B01N7PUAQ8_1, B01N7PUAQ8_2, B01N7PUAQ8_3, B06X3YL24L_2, B06X3YL24L_3, B06X3YL24L_6, B06X3YL24L_7, B0746KD3XW_4, B01GEH9IZU_1, B074FXGQP6_3, B01N6OH8JR_1, B01N6OH8JR_2, B01N6OH8JR_3, B00WN96QEA_4, B00WN96QEA_8, B071Y3B8KL_9, B01GLGGKD2_1, B01GLGGKD2_10, B06WGWZ7Z6_3, B06WGWZ7Z6_5, B076DBRSBD_7, B01LD0BPEU_2, B01LD0BPEU_6, B06WVPGQP7_4, B06WVPGQP7_7, B06XBD7J6D_10, B01N36UBBJ_8, B01HCJSRFG_6, B01HCJSRFG_9, B01D66ARIO_10, B01MZDE2S6_2, B01MZDE2S6_3, B01MZDE2S6_9, B01MU9NSCU_6, B01MU9NSCU_10, B0089IO47K_8, B009A7ZKGS_4, B009A7ZKGS_9, B009A7ZKGS_10, B00AWQMMGM_7, B00AWQMMGM_9, B0089IO7UO_6, B00I4IXBRY_1, B00I4IXBRY_2, B008DHZDCC_6, B007GE75HY_2, B00F3B33PU_1, B0089IOWE0_8, B0089IO43Y_7, B007OUBCRO_5, B015WJKQY8_6, B015WJKQY8_8, B06XVH8WXJ_2, B075SFRBS1_3, B01AW4GGLU_1, B01AW4GGLU_2, B01AW4GGLU_4, B01AW4GGLU_6, B01AW4GGLU_10, B00005RF5D_5, B00005RF5D_10,

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


 B00M4J2T7M_7, B017K0M7M6_4, B017K0M7M6_6, B017K0M7M6_9, B074V8TG6M_3, B074V8TG6M_7, B074V8TG6M_8, B00YJC9EIE_6, B00YJC9EIE_9, B007UHUM36_6, B007UHUM36_7, B00Y5EOHRY_9, B06XG1FHKX_5, B0179YAIRO_3, B0179YAIRO_10, B01H9OBFZ8_5, B013LIH7US_2, B013LIH7US_4, B013LIH7US_6, B002PEQKHG_1, B005Q4D7ZA_7, B00Z9Y4670_6, B01LZNRW0C_1, B01LZNRW0C_2, B01LZNRW0C_5, B006H07BA4_9, B00XND0WEU_10, B00ET0N3QK_6, B01JAD4NC8_3, B01JAD4NC8_9, B01M71S9DU_10, B005TQJS80_2, B01KVH709K_3, B00DZFALQ0_5, B013JQE56G_1, B013JQE56G_5, B019U7A2MY_3, B019U7A2MY_6, B01BHS53TG_8, B076F2HMP1_2, B076F2HMP1_4, B01FUIE9ZS_9, B075R7VPS9_2, B075R7VPS9_3, B00ECWE4WC_1, B00ECWE4WC_7, B0089IPOGK_5, B0089IPOGK_8, B01G0JVHSI_2, B01G0JVHSI_3, B01G0JVHSI_5, B009ZC9YBK_1, B009ZC9YBK_4, B06Y6DT7NL_1, B06Y6DT7NL_2, B00UINDM04_10, B009ZC964U_4, B009ZC964U_5, B009ZC964U_6, B009ZC964U_9, B01CCXZYVC_2, B01CCXZYVC_4, B01CCXZYVC_9, B01NBV4HSQ_3, B01NBV4HSQ_7, B01NBV4HSQ_8, B071GVSV3D_2, B01MDR83RN_10, B01JM4CGAQ_2, B06XDBDRSJ_3, B009RENV1U_3, 

Cluster 3 titles: B06Y3F148Q_1, B0746KMZ4W_1, B00SWURF62_9, B005HHRXWE_4, B005HHRXWE_8, B005HHRXWE_9, B01HV9XPN6_2, B01JC4680U_8, B074GR1WCX_1, B074GR1WCX_6, B074GR1WCX_9, B00RJE7MWE_3, B00RJE7MWE_5, B06X3YL24L_1, B06X3YL24L_4, B01L7688JU_8, B074FXGQP6_6, B00XJJGPWQ_1, B00XJJGPWQ_8, B071Y3B8KL_1, B06WGWZ7Z6_2, B06WGWZ7Z6_8, B076DBRSBD_1, B01D66ARIO_8, B01AW4GCJQ_1, B01AW4GCJQ_5, B01MU9NSCU_1, B0089IO47K_3, B0089IO47K_4, B00F3B2TVO_2, B00F3B2TVO_3, B00F3B2TVO_4, B009A7ZKGS_8, B06WVDSP61_1, B0089IO7UO_4, B0089IO7UO_7, B008DHZDCC_2, B007GE75HY_5, B007GE75HY_6, B007GE75HY_8, B009A7ZCB6_4, B009A7ZCB6_5, B009A7ZCB6_9, B00F3B33PU_5, B01L26UXQ6_5, B0089IOWE0_1, B0089IOWE0_10, B0089IO43Y_2, B0089IO43Y_3, B0089IO43Y_5, B007OUBCRO_2, B007OUBCRO_9, B075LDDQKH_2, B015WJKQY8_3, B015WJKQY8_5, B015WJKQY8_10, B007GE75G0_3, B0089IO0OW_1, B0089IO0OW_3, B0089IO0OW_4, B0089IO0OW_5, B0089IO0OW_7, B0089IO0OW_9, B00D3IMEUA_1, B0089IO4SO_2, B0089IO4SO_4, B0089IO4SO_9, B0089IO7OA_5, B0089IO7OA_9, B01MRVXMWP_2, 

Cluster 6 titles: B06Y3F148Q_6, B06Y3F148Q_9, B00SWURF62_1, B00SWURF62_3, B00SWURF62_6, B00SWURF62_8, B00RJE7MWE_7, B06X3YL24L_5, B06X3YL24L_9, B0746KD3XW_1, B01GEH9IZU_2, B071Y3B8KL_2, B071Y3B8KL_5, B06WP1WMHT_1, B01N25YA96_1, B00F3B2TVO_5, B00I4IXBRY_3, B007GE75HY_3, B007GE75HY_9, B007GE75HY_10, B00F3B33PU_3, B01L26UXQ6_1, B0089IOWE0_5, B007GE75G0_5, B007GE75G0_8, B06XVH8WXJ_6, B075SFRBS1_1, B075SFRBS1_2, B01MRVXMWP_4, B01MRVXMWP_6, B00006JZCG_5, B00006JZCG_6, B01MXWNQ9J_6, 1591747694_2, B00005RF5D_3, B00005RF5D_6, B00005RF5D_7, B01DIPS9GK_10, B0089IO3GW_10, B00A3QPCOA_2, B00A3QPCOA_4, B00A3QPCOA_9, B0177LV5SA_2, B00X9J1B4S_2, B00X9J1B4S_9, B075YZTQX4_5, B075YZTQX4_6, B075YZTQX4_7, B014DEHUU6_2, B014DEHUU6_3, B01CJKRA60_1, B074D4BS7Q_6, B074D4BS7Q_9, B013SVWB4U_2, B013SVWB4U_9, B07454YHRV_2, B01MFGS0MU_7, B014DEJF7W_2, B014DEJF7W_6, B014DEJF7W_10, B00K5NFGI8_2, B06XRWGS6Y_2, B008YD8GG0_1, B008YD8GG0_6, B01MTYDO5I_1, B01MTYDO5I_9, B01BKFFPF8_5, B01BKFFPF8_9, B01MUR1RSC_3, B01MUR1RSC_7

## 多维尺度分析（Multidimensional scaling）

In [ ]:
# import os  # 为了使用 os.path.basename 函数
 
# import matplotlib.pyplot as plt
# import matplotlib as mpl
 
# from sklearn.manifold import MDS
 
# MDS()
 
# # 将二位平面中绘制的点转化成两个元素（components）
# # 设置为“precomputed”是因为我们提供的是距离矩阵
# # 我们可以将“random_state”具体化来达到重复绘图的目的
# mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
 
# pos = mds.fit_transform(dist)  # 形如 (n_components, n_samples)
 
# xs, ys = pos[:, 0], pos[:, 1]

## 可视化文档聚类

In [ ]:
# #用字典设置每个聚类的颜色
# cluster_colors = {0: '#1c9e33', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e', 5: '#1b9c33', 6: '#d95f06',\
#                   7: '#7570b9', 8: '#e7288a', 9: '#66a63f'}
 
# # 用字典设置每个聚类名称
# cluster_names = {0: 'Kids, fun, pieces', 
#                  1: 'Kids, love, work', 
#                  2: 'Son, christmas, playing', 
#                  3: 'Christmas, bought, playing', 
#                  4: 'Months, old, playing',
#                  5: 'Grandsons, christmas, gift',
#                  6: 'Daughter, cute, work',
#                  7: 'Little, daughter, used',
#                  8: 'Christmas, daughter, playing',
#                  9: 'Christmas, pefect, bought'}

In [ ]:
# #在 ipython 中内联（inline）演示 matplotlib 绘图
# %matplotlib inline 
 
#  # 用 MDS 后的结果加上聚类编号和绘色创建 DataFrame
# df = pd.DataFrame(dict(x=xs, y=ys, label=clusters)) 
 
# # 聚类归类
# groups = df.groupby('label')
 
#  # 设置绘图
# fig, ax = plt.subplots(figsize=(17, 9)) # 设置大小
# ax.margins(0.05) # 可选项，只添加 5% 的填充（padding）来自动缩放（auto scaling）。
 
# # 对聚类进行迭代并分布在绘图上
# # 我用到了 cluster_name 和 cluster_color 字典的“name”项，这样会返回相应的 color 和 label
# for name, group in groups:
#     ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
#             label=cluster_names[name], color=cluster_colors[name], 
#             mec='none')
#     ax.set_aspect('auto')
#     ax.tick_params(
#         axis= 'x',          # 使用 x 坐标轴
#         which='both',      # 同时使用主刻度标签（major ticks）和次刻度标签（minor ticks）
#         bottom='off',      # 取消底部边缘（bottom edge）标签
#         top='off',         # 取消顶部边缘（top edge）标签
#         labelbottom='off')
#     ax.tick_params(
#         axis= 'y',         # 使用 y 坐标轴
#         which='both',      # 同时使用主刻度标签（major ticks）和次刻度标签（minor ticks）
#         left='off',      # 取消底部边缘（bottom edge）标签
#         top='off',         # 取消顶部边缘（top edge）标签
#         labelleft='off')
# ax.legend(numpoints=1)  # 图例（legend）中每项只显示一个点
 
# # 在坐标点为 x,y 处添加影片名作为标签（label）
# for i in range(len(df)):
#     ax.text(df.ix[i]['x'], df.ix[i]['y'],size=8)  
 
 
# plt.show() # 展示绘图
# # 以下注释语句可以保存需要的绘图
# #plt.savefig('clusters_small_noaxes.png', dpi=200)

In [ ]:
# plt.close()

## 文档层次聚类

In [ ]:
# from scipy.cluster.hierarchy import ward, dendrogram
 
# linkage_matrix = ward(dist) # 聚类算法处理之前计算得到的距离，用 linkage_matrix 表示
 
# fig, ax = plt.subplots(figsize=(15, 20)) # 设置大小
# ax = dendrogram(linkage_matrix, orientation="right", labels=ASINs);
 
# plt.tick_params(
#         axis= 'x',          # 使用 x 坐标轴
#         which='both',      # 同时使用主刻度标签（major ticks）和次刻度标签（minor ticks）
#         bottom='off',      # 取消底部边缘（bottom edge）标签
#         top='off',         # 取消顶部边缘（top edge）标签
#     labelbottom='off')
 
# plt.tight_layout() # 展示紧凑的绘图布局
 
# # 注释语句用来保存图片
# plt.savefig('ward_clusters_US20171130.png', dpi=200) # 保存图片为 ward_clusters

In [ ]:
# plt.close()

## LDA

In [13]:
# 去除文本中所有的专有名词……不幸的是，现在句子的第一个单词也被去掉了。
import string
def strip_proppers(text):
    #  首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

In [31]:
# 去除文本中所有专有名词（NNP）和复数名词（NNPS）
from nltk.tag import pos_tag
 
def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) # 使用 NLTK 的词性标注器
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [32]:
from gensim import corpora, models, similarities 
 
# 去除专有名词
%time preprocess = [strip_proppers(doc) for doc in review]
 
 # 分词
%time tokenized_text = [tokenize_and_stem(text) for text in preprocess]
 
 # 去停用词
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]


CPU times: user 3.56 s, sys: 33.7 ms, total: 3.59 s
Wall time: 3.78 s
CPU times: user 5.94 s, sys: 52.9 ms, total: 5.99 s
Wall time: 6.33 s
CPU times: user 521 ms, sys: 3.56 ms, total: 525 ms
Wall time: 526 ms


In [33]:
dic=[]
for list in texts:
    dic += list
dic = set(dic)
print(dic)

{'ho', 'today', 'shadow', 'slow', 'hubbi', "story'sand", 'toolbox', 'chockol', 'liter', 'put', 'redund', 'exchang', 'syring', 'healthi', 'bmx', 'rubber', '28cm', 'camper', 'faffi', 'well-received.|mi', 'car|noth', 'ie', 'pillow', 'consid', 'shave', 'sulphat', 'spider', 'decent', 'histori', 'video', 'attatch', 'abec', 'centr', 'suffer', 'rein', 'chair/bumbo', 'spanner', 'board.|skateboard', 'happy.easi', 'radioact', 'cooki', 'cupboard', 'au', 'f', 'yesterday', 'yr.', 'belt', 'four', 'plastik', 'trees/rock', 'wear', 'iron.|great', 'refund', 'spacious', 'otherwis', 'fantast', 'net', 'refil', 'graber', '🙃🙃🙃🙃|brilliant', 'off-complet', 'hood', '9year', 'cheapli', 'lax', 'carnt', 'ate', 'fair-siz', 'creativity.|purchas', 'duff', 'suppos', 'mishap', 'teacher/pupil', 'softi', 'todger', 'unscrew', 'nois', '5m', 'crimin', 'apply.nic', 'sharp', '6oz', 'tangerin', 'lip', 'mass', 'tv', 'recommended|high', 'elementari', 'soup', 'superhero', 'riding/pulling/carri', 'colleg', 'gas', 'else.|colour', 'g

In [34]:
with open('../UK.txt','w') as f:
    f.write(str(dic))

In [42]:
#print(len([word for word in texts[0] if word not in stopwords]))
print(len(texts[0]))

123


In [43]:
# 用文本构建 Gensim 字典
dictionary = corpora.Dictionary(texts)
 
# 去除极端的词（和构建 tf-idf 矩阵时用到 min/max df 参数时很像）
# dictionary.filter_extremes(no_below=1, no_above=0.8)
 
# 将字典转化为词典模型（bag of words）作为参考
corpus = [dictionary.doc2bow(text) for text in texts]


In [44]:
len(corpus)

961

In [45]:
%time lda = models.LdaModel(corpus, num_topics=10,id2word=dictionary,update_every=5,chunksize=10000,passes=100)

CPU times: user 6min 31s, sys: 2.8 s, total: 6min 33s
Wall time: 6min 38s


In [46]:
print(lda[corpus[0]])

[(2, 0.45735168), (6, 0.5361954)]


In [47]:
topics_matrix = lda.show_topics(formatted=False, num_words=100)

In [48]:
topics_matrix

[(0,
  [('cube', 0.040854666),
   ('good', 0.032115981),
   ('veri', 0.016724238),
   ('smooth', 0.015969051),
   ("'s", 0.014078646),
   ('product', 0.01228278),
   ('use', 0.011723666),
   ('one', 0.010157536),
   ('qualiti', 0.0095363231),
   ('price', 0.0094168084),
   ('buy', 0.0088923555),
   ('great', 0.0088769644),
   ('speed', 0.0084043937),
   ('like', 0.0066474564),
   ('sticker', 0.0064791795),
   ('get', 0.006433615),
   ('got', 0.0062899557),
   ('best', 0.0062051909),
   ('solv', 0.0061382786),
   ('color', 0.0057334322),
   ('also', 0.0056356951),
   ('realli', 0.0051830197),
   ('beginn', 0.0050300946),
   ("n't", 0.0050167432),
   ('much', 0.0049047666),
   ('nice', 0.0046104672),
   ('set', 0.0045248866),
   ('corner', 0.0041500665),
   ('origin', 0.0041060075),
   ('day', 0.0041039926),
   ('fast', 0.0040527009),
   ('bought', 0.0040156259),
   ('worth', 0.0037368126),
   ('go', 0.003724488),
   ('recommend', 0.0034234715),
   ('money', 0.0033366482),
   ('would', 0

In [49]:
A =[] 
for item in topics_matrix:
    a =[item[1]]
    A +=a

In [50]:
TM = np.array(A)
TM

array([[['cube', '0.0408547'],
        ['good', '0.032116'],
        ['veri', '0.0167242'],
        ..., 
        ['two', '0.00161224'],
        ['brand', '0.00157852'],
        ['start', '0.00157515']],

       [['kid', '0.0408585'],
        ['learn', '0.0174349'],
        ["'s", '0.0165474'],
        ..., 
        ['wonder', '0.00213031'],
        ['excel', '0.0021024'],
        ['even', '0.0020942']],

       [['good', '0.0467675'],
        ['product', '0.0440377'],
        ['qualiti', '0.0364117'],
        ..., 
        ['come', '0.0017177'],
        ['bought', '0.00171562'],
        ['beauti', '0.00171062']],

       ..., 
       [['product', '0.0477923'],
        ['good', '0.0334774'],
        ['qualiti', '0.0228619'],
        ..., 
        ['replac', '0.00192121'],
        ['help', '0.00190807'],
        ['hard', '0.00190167']],

       [['assembl', '0.0217498'],
        ['qualiti', '0.0203275'],
        ["n't", '0.0153005'],
        ..., 
        ['size', '0.0019754'],
        

In [51]:
voc = open('../vocab_US.txt').read().split('\t')
voc = set(voc)

In [52]:
topic_words = TM[:,:,0]
voc_list =[]
for i in topic_words:
    a = [str(word) for word in i if str(word) in voc]
    voc_list += [a]
voc_list

[['great',
  'speed',
  'best',
  'fast',
  'love',
  'expect',
  'plastic',
  'perfect',
  'cheap',
  'time',
  'brand'],
 ['love', 'time', 'fun', 'great', 'best', 'design'],
 ['love',
  'small',
  'size',
  'expect',
  'happi',
  'receiv',
  'great',
  'big',
  'plastic',
  'perfect',
  'time'],
 ['time',
  'love',
  'great',
  'happi',
  'size',
  'expect',
  'perfect',
  'small',
  'fun',
  'receiv',
  'cheap'],
 ['plastic',
  'love',
  'best',
  'great',
  'big',
  'seller',
  'perfect',
  'happi',
  'expect',
  'brand'],
 ['time', 'size', 'fun', 'small', 'expect', 'perfect', 'soon', 'hit', 'love'],
 ['perfect',
  'design',
  'best',
  'time',
  'detail',
  'review',
  'size',
  'love',
  'great',
  'rough'],
 ['love',
  'small',
  'time',
  'plastic',
  'expect',
  'receiv',
  'size',
  'happi',
  'cheap'],
 ['expect', 'detail', 'happi', 'size', 'design'],
 ['time',
  'receiv',
  'small',
  'cheap',
  'love',
  'happi',
  'expect',
  'great',
  'best']]

In [ ]:
#title
asins = open('../US_20171130/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
len(ASINs)

In [5]:
dic = []
US = open('../US.txt').read().split(',')
IN = open('../IN.txt').read().split(',')  
UK = open('../UK.txt').read().split(',')  
dic += US
dic += IN
dic += UK
dic = set(dic)

In [3]:
with open

"{'laura'"